In [42]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt 
import seaborn as sns

# Disable annoying warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

from sklearn.dummy import DummyClassifier
from sklearn import metrics

In [43]:
df = pd.read_csv("nba_player_data.csv")
df = df.dropna()
df

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,Arron Afflalo,SG,24,DEN,82,75,27.1,3.3,7.1,0.465,...,0.7,2.4,3.1,1.7,0.6,0.4,0.9,2.7,8.8,2010
2,LaMarcus Aldridge,PF,24,POR,78,78,37.5,7.4,15.0,0.495,...,2.5,5.6,8.0,2.1,0.9,0.6,1.3,3.0,17.9,2010
3,Joe Alexander,SF,23,CHI,8,0,3.6,0.1,0.8,0.167,...,0.3,0.4,0.6,0.3,0.1,0.1,0.0,1.1,0.5,2010
4,Malik Allen,PF,31,DEN,51,3,8.9,0.9,2.3,0.397,...,0.7,0.9,1.6,0.3,0.2,0.1,0.4,1.3,2.1,2010
5,Ray Allen*,SG,34,BOS,80,80,35.2,5.8,12.2,0.477,...,0.6,2.6,3.2,2.6,0.8,0.3,1.6,2.3,16.3,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8296,Thaddeus Young,PF,33,SAS,26,1,14.2,2.8,4.9,0.578,...,1.5,2.0,3.6,2.3,0.9,0.3,1.2,1.5,6.1,2022
8297,Thaddeus Young,PF,33,TOR,26,0,18.3,2.6,5.5,0.465,...,1.5,2.9,4.4,1.7,1.2,0.4,0.8,1.7,6.3,2022
8298,Trae Young,PG,23,ATL,76,76,34.9,9.4,20.3,0.460,...,0.7,3.1,3.7,9.7,0.9,0.1,4.0,1.7,28.4,2022
8299,Omer Yurtseven,C,23,MIA,56,12,12.6,2.3,4.4,0.526,...,1.5,3.7,5.3,0.9,0.3,0.4,0.7,1.5,5.3,2022


In [44]:
y = df["PTS"]
X = df.drop("PTS", axis=1)
X = X.drop("Player", axis=1)
X = X.drop("Year", axis=1)
X = X.drop("Pos", axis=1)
X = X.drop("Tm", axis=1)

X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size = 0.3, random_state = 0)

In [45]:
strategies = ['most_frequent', 'stratified', 'uniform']

test_scores = []
for s in strategies:
    dclf = DummyClassifier(strategy = s)
    dclf.fit(X_train, y_train)
    accuracy = metrics.accuracy_score(dclf.predict(X_train), y_train)
    
    #print(dclf.class_prior_)
    print(accuracy)
    # score = dclf.score(X_test, y_test)
    # test_scores.append(score)

ValueError: Classification metrics can't handle a mix of binary and continuous targets